In [1]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [1]:
import os
import requests
import pandas as pd
import sqlite3

In [4]:
# download dos arquivos
# url_base para os datasets do IMDB
base_url="https://datasets.imdbws.com/"

# namepath dos arquivos que desejamos baixar
files = [
    "name.basics.tsv.gz",
    "title.akas.tsv.gz",
    "title.basics.tsv.gz",
    "title.crew.tsv.gz",
    "title.episode.tsv.gz",
    "title.principals.tsv.gz",
    "title.ratings.tsv.gz"
]

# diretório destino
base_dir = "Data"

# Certifique-se de que o diretório destino existe
os.makedirs(base_dir, exist_ok=True)

#Loop para baixar os arquivos
for filename in files:
    url = base_url + filename
    namepath = os.path.join(base_dir, filename)
    
    # verifica se o arquivo já existe para evitar o download repetido
    if not os.path.exists(namepath):
        print(f"Baixando {filename}...")
        response = requests.get(url)
        
        # verifica se a solicitação foi bem sucedida
        if response.status_code == 200:
            with open(namepath, 'wb') as f:
                f.write(response.content)
            print(f"{filename} baixado com sucesso")
        else:
            print(f"Falha ao baixar {filename}. Código de status: {response.status_code}")
    else:
        print(f"{filename} já existe.")
print(f"Download concluído.")

name.basics.tsv.gz já existe.
title.akas.tsv.gz já existe.
title.basics.tsv.gz já existe.
title.crew.tsv.gz já existe.
title.episode.tsv.gz já existe.
title.principals.tsv.gz já existe.
title.ratings.tsv.gz já existe.
Download concluído.


In [6]:
# Tratamento dos dados
processed_dir = os.path.join(base_dir, "processed")

# Certifique-se de que o diretório processed existe
os.makedirs(processed_dir, exist_ok=True)

# Lista todos os arquivos do diretório Data
files = os.listdir(base_dir)

# Loop para abrir, tratar e salvar cada arquivo
for filename in files:
    namepath = os.path.join(base_dir, filename)
    
    if os.path.isfile(namepath) and filename.endswith("gz"):
        print(f"Lendo e tratando arquivo {filename}...")
        
        # Lê o arquivo TSV usando pandas
        df = pd.read_csv(namepath, sep='\t', compression='gzip', low_memory=False)
        
        # Substitui os caracteres "ln" um valor nulo
        df.replace({"\\N": None}, inplace=True)
        
        # Salva o DataFrame no diretório "processed" sem compressão
        output_file_path = os.path.join(processed_dir, filename[:-3])  # Caminho completo para o arquivo processado
        df.to_csv(output_file_path, sep='\t', index=False)
        
        print(f"Tratamento concluído para {filename}. Arquivo tratado e salvo em {output_file_path}")
print(f"Todos os arquivos foram tratados e salvos no diretório Processed")

Lendo e tratando arquivo name.basics.tsv.gz...
Tratamento concluído para name.basics.tsv.gz. Arquivo tratado e salvo em Data\processed\name.basics.tsv
Lendo e tratando arquivo title.akas.tsv.gz...
Tratamento concluído para title.akas.tsv.gz. Arquivo tratado e salvo em Data\processed\title.akas.tsv
Lendo e tratando arquivo title.basics.tsv.gz...
Tratamento concluído para title.basics.tsv.gz. Arquivo tratado e salvo em Data\processed\title.basics.tsv
Lendo e tratando arquivo title.crew.tsv.gz...
Tratamento concluído para title.crew.tsv.gz. Arquivo tratado e salvo em Data\processed\title.crew.tsv
Lendo e tratando arquivo title.episode.tsv.gz...
Tratamento concluído para title.episode.tsv.gz. Arquivo tratado e salvo em Data\processed\title.episode.tsv
Lendo e tratando arquivo title.principals.tsv.gz...
Tratamento concluído para title.principals.tsv.gz. Arquivo tratado e salvo em Data\processed\title.principals.tsv
Lendo e tratando arquivo title.ratings.tsv.gz...
Tratamento concluído para t

In [32]:
# Validando dataframe
df = pd.read_csv('./Data/processed/name.basics.tsv', sep='\t')
df.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0043044"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0050986,tt0069467,tt0050976,tt0083922"


In [33]:
# Verifica a quantidade de dados nulos por coluna
df.isnull().sum()

nconst                      0
primaryName                59
birthYear            13389443
deathYear            13785332
primaryProfession     2723305
knownForTitles        1587326
dtype: int64

In [34]:
# Salvando em banco de dados com o SQLite

# Diretórios
processed_dir = os.path.join("Data", "processed")
db = "imdb_data.db"

# Conecta ao banco de dados SQLite
conn = sqlite3.connect(db)

# Lista todos os arquivos no diretório processed
files = os.listdir(processed_dir)

# Loop para ler cada arquivo e salvar em uma tabela SQLite
for filename in files:
    output_file_path = os.path.join(processed_dir, filename)

    if os.path.isfile(output_file_path) and filename.endswith(".tsv"):
        # Lê o arquivo tsv usando o pandas
        df = pd.read_csv(output_file_path, sep='\t', low_memory=False)

        # O nome da tabela é o nome do arquivo sem a extensão do nome
        table_name = os.path.splitext(filename)[0]

        # Substitui os caracteres especiais no nome da tabela
        table_name = table_name.replace(".", "_").replace("-", "_")

        # Salva o Dataframe na tabela SQLite
        df.to_sql(table_name, conn, index=False, if_exists='replace')

        print(f"Aquivo {filename} salvo como tabela {table_name} no banco de dados")

# Fecha a conexão com o banco de dados
conn.close()
print("Todos os arquivos foram salvos no banco de dados.")

Aquivo name.basics.tsv salvo como tabela name_basics no banco de dados
Aquivo title.akas.tsv salvo como tabela title_akas no banco de dados
Aquivo title.basics.tsv salvo como tabela title_basics no banco de dados
Aquivo title.crew.tsv salvo como tabela title_crew no banco de dados
Aquivo title.episode.tsv salvo como tabela title_episode no banco de dados
Aquivo title.principals.tsv salvo como tabela title_principals no banco de dados
Aquivo title.ratings.tsv salvo como tabela title_ratings no banco de dados
Todos os arquivos foram salvos no banco de dados.


In [85]:
# Teste de conexão: Exibir schemas

# Nome do banco de dados
db = "imdb_data.db"

# Conecta-se ao banco de dados SQLite
conn = sqlite3.connect(db)

# Cria um cursor para validar as queries
cursor = conn.cursor()

# Executa a consulta SQL para obter o nome das tabelas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Recupera os resultados da consulta
tables = cursor.fetchall()

# Exibe o nome das tabelas
print("Tabelas no banco de dados:")
for t in tables:
    print(t[0])

# Fecha o cursor e a conexão com o banco de dados
cursor.close()
conn.close()

Tabelas no banco de dados:
name_basics
title_akas
title_basics
title_crew
title_episode
title_principals
title_ratings
table_analytics
table_participants


In [36]:
# Teste de conexão: Exibir conteúdo de tabela

# Conecta-se com o banco de dados
conn = sqlite3.connect(db)

# Executa a consulta SQL para obter as 10 primeiras linhas da tabela 
query = """
SELECT * FROM title_basics LIMIT 10;
"""
df = pd.read_sql_query(query, conn)

# Fecha a conexão com o banco de dados
conn.close()

In [79]:
print(df)

      tconst  ordering         category                    genres
0  tt0000001         1             self         Documentary,Short
1  tt0000001         2         director         Documentary,Short
2  tt0000001         3         producer         Documentary,Short
3  tt0000001         4  cinematographer         Documentary,Short
4  tt0000002         1         director           Animation,Short
5  tt0000002         2         composer           Animation,Short
6  tt0000003         1         director  Animation,Comedy,Romance
7  tt0000003         2         producer  Animation,Comedy,Romance
8  tt0000003         3         producer  Animation,Comedy,Romance
9  tt0000003         4         composer  Animation,Comedy,Romance


In [78]:
# Conecta-se com o banco de dados
conn = sqlite3.connect(db)

# Executa a consulta SQL para obter as 10 primeiras linhas da tabela 
"""
    SELECT
        startYear,
        count(startYear)
    FROM title_basics 
    GROUP BY(startYear)
"""
"""
    SELECT
        tb.tconst,
        tb.titleType,
        tb.originalTitle,
        tb.startYear,
        tb.genres,
        tr.averageRating,
        tr.numVotes 
    FROM title_basics tb
    LEFT JOIN title_ratings tr ON tb.tconst = tr.tconst
    LIMIT 10;
"""

"""
    WITH
        participants AS (
            SELECT
                    tp.tconst,
                    COUNT (DISTINCT tp.nconst) as QtParticipants
                FROM title_principals tp
                GROUP BY 1
        )
    SELECT
        tb.tconst,
        tb.titleType,
        tb.originalTitle,
        tb.startYear,
        tb.endYear,
        tb.genres,
        tr.averageRating,
        tr.numVotes,
        tp.QtParticipants
    FROM title_basics tb
    LEFT JOIN title_ratings tr     ON tb.tconst = tr.tconst
    LEFT JOIN participants tp      ON tp.tconst = tb.tconst

    LIMIT 10;
"""

query = """
SELECT 
    tb.tconst,
    tp.ordering,  
    tp.category,                         
    tb.genres
FROM title_principals tp
LEFT JOIN title_basics tb   ON tp.tconst = tb.tconst
LIMIT 10;
"""

df = pd.read_sql_query(query, conn)

# Fecha a conexão com o banco de dados
conn.close()

In [81]:
query1 = """
CREATE TABLE IF NOT EXISTS table_analytics AS
    WITH
        participants AS (
            SELECT
                    tp.tconst,
                    COUNT (DISTINCT tp.nconst) as QtParticipants
                FROM title_principals tp
                GROUP BY 1
        )
    SELECT
        tb.tconst,
        tb.titleType,
        tb.originalTitle,
        tb.startYear,
        tb.endYear,
        tb.genres,
        tr.averageRating,
        tr.numVotes,
        tp.QtParticipants
    FROM title_basics tb
    LEFT JOIN title_ratings tr     ON tb.tconst = tr.tconst
    LEFT JOIN participants tp      ON tp.tconst = tb.tconst
"""

query2 = """
CREATE TABLE IF NOT EXISTS table_participants AS
    SELECT 
        tb.tconst,
        tp.ordering,  
        tp.category,                         
        tb.genres
    FROM title_principals tp
    LEFT JOIN title_basics tb   ON tp.tconst = tb.tconst
"""

In [ ]:
# Lista de consultas
queries = [query1, query2]
for q in queries:
    # Diretórios
    bd = "imdb_data.db"
    
    # Conecta-se com o banco de dados
    conn = sqlite3.connect(db)

    # Execução
    conn.execute(q)
# Fecha a conexão
conn.close()
print("Tabelas criadas com sucesso!")
    

Tabelas criadas com sucesso!


In [5]:
# Conecta-se com o banco de dados
db = "imdb_data.db"
conn = sqlite3.connect(db)

# Consulta
query = """
SELECT 
    *
FROM table_participants
LIMIT 10;
"""

df = pd.read_sql_query(query, conn)

# Fecha a conexão com o banco de dados
conn.close()
print(df)


      tconst  ordering         category                    genres
0  tt0000001         1             self         Documentary,Short
1  tt0000001         2         director         Documentary,Short
2  tt0000001         3         producer         Documentary,Short
3  tt0000001         4  cinematographer         Documentary,Short
4  tt0000002         1         director           Animation,Short
5  tt0000002         2         composer           Animation,Short
6  tt0000003         1         director  Animation,Comedy,Romance
7  tt0000003         2         producer  Animation,Comedy,Romance
8  tt0000003         3         producer  Animation,Comedy,Romance
9  tt0000003         4         composer  Animation,Comedy,Romance
